In [ ]:
import yfinance as yf
import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, losses
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from pathlib import Path
from loguru import logger
from concurrent.futures import ProcessPoolExecutor
from scipy.stats import skew, kurtosis

from src.universe import Universe
from src.utils import time_series_generator
from src.metrics import plot_mse, generate_report
from src.model.deepar import DeepAR

In [ ]:
inception_date = '2022-03-31'
ONE_YEAR_TRADE_DAYS = 252
TWO_YEAR_TRADE_DAYS = ONE_YEAR_TRADE_DAYS * 2
WIN_LEN = 30
UNIVERSE_SIZE = 2500
EPOCH = 20
BATCH_SIZE = 16

In [ ]:
universe = Universe()

In [ ]:
liquid_ticker_ret = universe.get_liquid_ticker_return('2022-03-31')

In [ ]:
ret_train = liquid_ticker_ret.loc[:inception_date].iloc[-TWO_YEAR_TRADE_DAYS-WIN_LEN:]
train_index = ret_train.loc[:inception_date].iloc[-TWO_YEAR_TRADE_DAYS:].index
train_flat = ret_train.values.flatten()
train_flat = train_flat[train_flat != 0]
m = train_flat.mean()
s = train_flat.std()
train_flat = train_flat.clip(m-2*s, m+2*s)
sns.kdeplot(train_flat)

In [ ]:
ret_train = ret_train.clip(m-2*s, m+2*s)
X, y = time_series_generator(ret_train, WIN_LEN)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X,y)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
deepar = DeepAR(WIN_LEN, UNIVERSE_SIZE)

In [ ]:
deepar.build_and_compile()

In [ ]:
deepar.summary()

In [ ]:
esc = tf.keras.callbacks.EarlyStopping(
    monitor='val_mse',
    patience=2,
    mode='auto',
)
deepar.fit(
    x=X_train, y=y_train, validation_data=(X_val, y_val), 
    epochs=EPOCH, batch_size=BATCH_SIZE,
    callbacks=[esc]
    )

In [ ]:
## get ticker returns
test_start_date = liquid_ticker_ret.loc[:inception_date].iloc[-WIN_LEN:].index[0]
ret_test = liquid_ticker_ret.loc[test_start_date:].iloc[:ONE_YEAR_TRADE_DAYS + WIN_LEN+1]
ret_test = ret_test.clip(m-2*s, m+2*s)
X_test, y_test = time_series_generator(ret_test, WIN_LEN)

In [ ]:
test_index = ret_test.loc[inception_date:].iloc[1:ONE_YEAR_TRADE_DAYS].index
test_index

In [ ]:
y_pred_train_mu = deepar.predict_mu(X)
y_pred_test_mu = deepar.predict_mu(X_test)

y_pred_train_rand = deepar.predict_rand(X)
y_pred_test_rand = deepar.predict_rand(X_test)

In [ ]:
ax_train, mse_df_train = plot_mse([y_pred_train_mu, y_pred_train_rand], y, ['deepar_mu', 'deepar_rand'], train_index)

In [ ]:
ax_test, mse_df_test = plot_mse([y_pred_test_mu, y_pred_test_rand], y_test, ['deepar_mu', 'deepar_rand'], test_index)

In [ ]:
generate_report([y_pred_test_mu, y_pred_test_rand], y_test, ['deepar_mu', 'deepar_rand'])